In [14]:
print("This all project about the Gender and age detection using openCV")

This all project about the Gender and age detection using openCV


In [2]:
import numpy

In [3]:
pip install opencv-python numpy


   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB 653.6 kB/s eta 0:01:01
   ---------------------------------------- 0.1/39.5 MB 1.1 MB/s eta 0:00:37
   ---------------------------------------- 0.2/39.5 MB 1.1 MB/s eta 0:00:38
   ---------------------------------------- 0.3/39.5 MB 1.3 MB/s eta 0:00:31
   ---------------------------------------- 0.3/39.5 MB 1.3 MB/s eta 0:00:31
   ---------------------------------------- 0.4/39.5 MB 1.4 MB/s eta 0:00:28
   ---------------------------------------- 0.5/39.5 MB 1.3 MB/s eta 0:00:30
    --------------------------------------- 0.6/39.5 MB 1.4 MB/s eta 0:00:29
    --------------------------------------- 0.6/39.5 MB 1.4 MB/s eta 0:00:28
    --------------------------------------- 0.7/39.5 MB 1.5 MB/s eta 0:00:27
    --------------------------------------- 0.8/39.5 MB 1.4 MB/s eta 0:00:27
    -------


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import cv2
import numpy as np

# Paths to model files
faceProto = "opencv_face_detector.pbtxt"
faceModel = "opencv_face_detector_uint8.pb"

ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"

genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"

# Load networks
faceNet = cv2.dnn.readNet(faceModel, faceProto)
ageNet = cv2.dnn.readNet(ageModel, ageProto)
genderNet = cv2.dnn.readNet(genderModel, genderProto)

# Define age and gender lists
AGE_LIST = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
GENDER_LIST = ['Male', 'Female']
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

# Read sample image
image = cv2.imread("sample1.jpg")
frame = image.copy()

# Face detection
blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), [104, 117, 123], swapRB=False)
faceNet.setInput(blob)
detections = faceNet.forward()

h, w = frame.shape[:2]
for i in range(detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    if confidence > 0.7:
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        x1, y1, x2, y2 = box.astype(int)

        # Extract face region
        face = frame[y1:y2, x1:x2]
        if face.size == 0:
            continue

        face_blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

        # Predict Gender
        genderNet.setInput(face_blob)
        genderPred = genderNet.forward()
        gender = GENDER_LIST[genderPred[0].argmax()]

        # Predict Age
        ageNet.setInput(face_blob)
        agePred = ageNet.forward()
        age = AGE_LIST[agePred[0].argmax()]

        label = f"{gender}, {age}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 2)
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

# Show image
cv2.imshow("Age and Gender Prediction", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [5]:
import cv2
import numpy as np

# Paths to model files
faceProto = "opencv_face_detector.pbtxt"
faceModel = "opencv_face_detector_uint8.pb"

ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"

genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"

# Load networks
faceNet = cv2.dnn.readNet(faceModel, faceProto)
ageNet = cv2.dnn.readNet(ageModel, ageProto)
genderNet = cv2.dnn.readNet(genderModel, genderProto)

# Define age and gender lists
AGE_LIST = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
GENDER_LIST = ['Male', 'Female']
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), [104, 117, 123], swapRB=False)
    faceNet.setInput(blob)
    detections = faceNet.forward()

    h, w = frame.shape[:2]
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.7:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            x1, y1, x2, y2 = box.astype(int)

            face = frame[y1:y2, x1:x2]
            if face.size == 0:
                continue

            face_blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

            genderNet.setInput(face_blob)
            genderPred = genderNet.forward()
            gender = GENDER_LIST[genderPred[0].argmax()]

            ageNet.setInput(face_blob)
            agePred = ageNet.forward()
            age = AGE_LIST[agePred[0].argmax()]

            # Custom age relabeling
            custom_age = age
            if age in ['(8-12)', '(15-20)']:
                custom_age = '10-20'
            elif age in ['(15-20)', '(25-32)']:
                custom_age = '20-30'

            label = f"{gender}, {custom_age}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    cv2.imshow("Age and Gender Prediction", frame)

    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 